In [4]:
import os
import s3fs
import gzip
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd

os.environ["AWS_ACCESS_KEY_ID"] = 'BFMNK1LUPXQ061C6PZ55'
os.environ["AWS_SECRET_ACCESS_KEY"] = '1lUWhLj95YhcD8P5t6GEMXxwlf0jyzmqrGPMURQE'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJCRk1OSzFMVVBYUTA2MUM2UFo1NSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzU5MDQ2Njc3LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzU5NjUxNTE1LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc1OTA0NjcxNSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDplNzkxZWViNS1kYWQyLWIxZjYtMzJjZi05YTM2ODVjNTNlNGIiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6ImFmY2VhMjNmLTM3MGUtNDhjNC1hNGQ5LWIzZDJjMDJkYmNkOCIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.JF4l6bNSZ2jJniDyrrDJRipM1GFJh5a05vfMLSwOcL5vXJbg6BjsZ639IP1NSk2phHnYPPnpD8xnCk6Pub1mEw'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

path = "lab/dblp (1).xml.gz"
%pip install lxml


Note: you may need to restart the kernel to use updated packages.


In [ ]:
articles = []

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        #parser = etree.XMLParser(recover=True, resolve_entities=True)
        context = etree.iterparse(gz, events=("end",), tag="article", recover=True)
        
        for event, elem in context:
            title = elem.findtext("title")
            journal = elem.findtext("journal")
            pub_date = elem.findtext("year")

            authors = [a.text for a in elem.findall(".//author")]
            authors_str = "; ".join([a for a in authors if a])

            articles.append({
                "title": title,
                "journal": journal,
                "year": pub_date,
                "authors": authors_str
            })

            elem.clear()

df = pd.DataFrame(articles)

Note: you may need to restart the kernel to use updated packages.


: 

In [5]:
conf = []

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        #parser = etree.XMLParser(recover=True, resolve_entities=True)
        context = etree.iterparse(gz, events=("end",), tag="inproceedings", recover=True)
        
        for event, elem in context:
            title = elem.findtext("title")
            journal = elem.findtext("journal")
            if journal == None : 
                journal = elem.findtext("crossref")
            pub_date = elem.findtext("year")

            authors = [a.text for a in elem.findall(".//author")]
            authors_str = "; ".join([a for a in authors if a])

            conf.append({
                "title": title,
                "journal": journal,
                "year": pub_date,
                "authors": authors_str
            })

            elem.clear()

df = pd.DataFrame(conf)

In [6]:
df

,title,journal,year,authors
0,The Future of Classic Data Administration: Obj...,conf/swee/1998,1998,Arnon Rosenthal
1,Some Patterns of Convincing Software Engineeri...,books/sp/22/FHKMPRRS2022,2020,Lutz Prechelt
2,Static Worst-Case Analyses and Their Validatio...,books/sp/22/FHKMPRRS2022,2020,Peter Wgemann
3,Crossing Disciplinary Borders to Improve Requi...,books/sp/22/FHKMPRRS2022,2020,Anne Hess
4,What You See Is What You Get: Practical Effect...,books/sp/22/FHKMPRRS2022,2020,Jonathan Immanuel Brachthuser
...,...,...,...,...
3749511,"Toward a Modern Geography of Minds, Machines, ...",series/sapere/2013-5,2011,Selmer Bringsjord; Naveen Sundar Govindarajulu
3749512,Emotional Control-Conditio Sine Qua Non for Ad...,series/sapere/2013-5,2011,Claudius Gros
3749513,Becoming Digital: Reconciling Theories of Digi...,series/sapere/2013-5,2011,Harry Halpin
3749514,Generative Artificial Intelligence.,series/sapere/2013-5,2011,Tijn van der Zant; Matthijs Kouw; Lambert Scho...


In [11]:
direct_set = set()  # pour stocker tous les noms de sous-éléments

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        context = etree.iterparse(gz, events=("end",), tag="inproceedings", recover=True)
        for event, elem in context:
            for child in elem:
                direct_set.add(child.tag)
            elem.clear()

print(sorted(direct_set))

: 

In [5]:
chunks = []
n_chunks = 50
chunk_size = len(df) // n_chunks

for i in range(n_chunks):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < n_chunks - 1 else len(df)  # last chunk includes remainder
    
    chunk = df.iloc[start:end].copy()
    
    # Split authors column into multiple columns
    #authors_df = chunk['authors'].str.split(';', expand=True)
    #authors_df.columns = [f'author{i+1}' for i in range(authors_df.shape[1])]
    
    # Combine with original chunk
    #chunk = pd.concat([chunk.drop(columns=['authors']), authors_df], axis=1)
    
    chunk.to_parquet(f"t/articles_chunk_{i}.parquet", index=False)

In [13]:

import pandas as pd
from collections import defaultdict
import glob

ego_data = defaultdict(lambda: {"publications": []}) #set permet de créer des clefs si absentes et de les actualiser si présentes (en définissant la classe des clefs)

chunk_files = glob.glob("t/articles_chunk_*.parquet")

for file in chunk_files:
    df_chunk = pd.read_parquet(file)
    """
    for _, row in df_chunk.iterrows():
        authors = row['authors'].split(';')
        for ego in authors:
            coauthors = set(authors) - {ego}
            #ego_data[ego]["coauthors"].update(coauthors)
            
            ego_data[ego]["covariates"].update({
                "coauthors" : list(coauthors),
                "journal": row.get("journal"),
                "year": row.get("year"),
                "title" : row.get("title")
            })
    """
    for _, row in df_chunk.iterrows():
        authors = row['authors'].split(';')
        for ego in authors:
            coauthors = set(authors) - {ego}
            
            ego_data[ego]["publications"].append({
                "coauthors": list(coauthors),
                "journal": row.get("journal"),
                "year": row.get("year"),
                "title": row.get("title")
            })

In [15]:
"""ego_df = pd.DataFrame([
    {'ego': ego, **data["publications"]} #récup paries ego - alter et trie dans l'ordre alphabétique
    for ego, data in ego_data.items()
])"""
rows = []
for ego, data in ego_data.items():
    for pub in data["publications"]:
        rows.append({
            "ego": ego,
            "coauthors": ",".join(sorted(pub["coauthors"])),
            "journal": pub.get("journal"),
            "year": pub.get("year"),
            "title": pub.get("title")
        })

ego_df = pd.DataFrame(rows)


In [24]:
ego_df

,ego,coauthors,journal,year,title
0,Baogang Xu,Haihui Zhang,Discret. Appl. Math.,2007,Every toroidal graph without adjacent triangle...
1,Baogang Xu,Lusheng Wang 0001,Discret. Appl. Math.,2005,Decomposing toroidal graphs into circuits and ...
2,Baogang Xu,Xingxing Yu,Graphs Comb.,2012,Maximum Directed Cuts in Graphs with Degree Co...
3,Baogang Xu,Xingxing Yu,Comb. Probab. Comput.,2011,Better Bounds for
4,Baogang Xu,,Discret. Math. Algorithms Appl.,2009,A Note on 3-colorable Plane Graphs without 5- ...
...,...,...,...,...,...
13845993,John Bracken,"Arundhuti Ganguly, Bruce Lewis Daniel, Daniel...",Proc. IEEE,2008,"Design, Performance, and Applications of a Hyb..."
13845994,Zhifei Wen,"Arundhuti Ganguly, Bruce Lewis Daniel, Daniel...",Proc. IEEE,2008,"Design, Performance, and Applications of a Hyb..."
13845995,Huanzhou Yu,"Arundhuti Ganguly, Bruce Lewis Daniel, Daniel...",Proc. IEEE,2008,"Design, Performance, and Applications of a Hyb..."
13845996,Viola Rieke,"Arundhuti Ganguly, Bruce Lewis Daniel, Daniel...",Proc. IEEE,2008,"Design, Performance, and Applications of a Hyb..."


In [ ]:
ego_df.to_parquet("ego_coauthors.parquet", index=False)
s3_path = "s3://lab/ego_coauthors.parquet"  # bucket + key

# Save directly to MinIO
ego_df.to_parquet(s3_path, engine="pyarrow", index=False, filesystem=fs)

In [8]:
ego_df.to_parquet("ego_df.parquet",index=False)
